In [1]:
import os
import shutil
from PIL import Image
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
import ffmpeg
import subprocess

In [2]:
path = os.path.dirname(os.path.dirname(os.path.abspath('distance_calculation.ipynb')))
print(path)
template_data_df = pd.read_csv(f'{path}/dataset_generator/movie_timelines/WIZARD OF OZ (1939)_complete_data.csv')
#template_data_df = pd.read_csv(f'{path}/dataset_generator/movie_timelines/2001_A_SPACE_ODYSSEY_complete_data.csv')
print(template_data_df.head())

C:\Users\Simon\git\Machine-Learning-And-Art
     (0, 0, 0)  (0, 0, 128)  (0, 0, 255)  (0, 128, 0)  (0, 128, 128)  \
0  3867.343895  3637.546540  2037.422507  4896.568180    4666.770825   
1  5210.163760  3581.850218  2624.749928  4954.966738    3326.653196   
2  7048.486982  4695.823260  4081.579478  5771.219120    3418.555397   
3  5866.305614  4145.388458  3291.526567  5362.353202    3641.436045   
4  3983.606436  2993.471874  1869.832447  4616.345162    3626.210600   

   (0, 128, 255)  (0, 255, 0)  (0, 255, 128)  (0, 255, 255)  (128, 0, 0)  ...  \
0    3066.646792  2628.763393    2398.966039     798.842006  5978.001755  ...   
1    2369.552906  3406.270174    1777.956632     820.856342  6820.053179  ...   
2    2804.311616  4656.433902    2303.770180    1689.526398  7210.946376  ...   
3    2787.574154  3962.345457    2241.428300    1387.566409  6695.390486  ...   
4    2502.571173  2723.869687    1733.735125     610.095698  6256.891533  ...   

   end_frame    end_time            

In [3]:
sample_data_df = pd.read_csv(f'{path}/dataset_generator/movie_timelines/SD_complete_data.csv')
#sample_data_df = pd.read_csv(f'{path}/dataset_generator/timeline_data/WIZARD OF OZ (1939)_complete_data.csv')
print(sample_data_df.head())
#print(sample_data_df.columns[35])

     (0, 0, 0)  (0, 0, 128)  (0, 0, 255)  (0, 128, 0)  (0, 128, 128)  \
0  1038.033050  3223.163146  1178.474253  3473.649198    5658.779294   
1  3563.952147  4974.373433  3666.370263  3700.360567    5110.781853   
2  2575.560373  2613.510912  3611.999700  3503.139958    3541.090497   
3  1091.764395  4144.522055   807.549576  4097.249362    7150.007022   
4  1151.272626  3279.617260   825.218506  3435.750458    5564.095092   

   (0, 128, 255)  (0, 255, 0)  (0, 255, 128)  (0, 255, 255)  (128, 0, 0)  ...  \
0    3614.090401  1133.973891    3319.103987    1274.415094  3680.403651  ...   
1    3802.778683  2468.915965    3879.337250    2571.334081  3738.993535  ...   
2    4539.579285  2074.898400    2112.848940    3111.337727  4214.268036  ...   
3    3813.034543   701.282849    3754.040509     417.068030  4106.544429  ...   
4    3109.696339  1095.964353    3224.308988     769.910234  3853.949323  ...   

   object_id_71  object_id_72  object_id_73  object_id_74  object_id_75  \
0    

In [4]:
def distance_calculator(expected_frame, possible_frame):
    color_difference = 0
    for i in range(27):
        color_difference += abs(expected_frame[i] - possible_frame[i])
    person_count_match_bonus = -5000
    person_object_id = 35
    if expected_frame[person_object_id] == possible_frame[person_object_id]:
        color_difference += person_count_match_bonus
    return color_difference

In [5]:
def best_frame_finder(expected_frame, possible_frames_df, best_frames=5, auto_sort=False, use_threshhold=False, threshhold=15000):
    possible_frame_matrix = possible_frames_df.to_numpy()
    possible_frame_cont = len(possible_frame_matrix)
    frame_scores = []
    for i in range(possible_frame_cont):
        frame_scores.append([i, distance_calculator(expected_frame, possible_frame_matrix[i])])
        
    if auto_sort:
        frame_scores = sorted(frame_scores, key=lambda x: x[1])
    if best_frames < len(possible_frames_df):
        frame_scores = frame_scores[0:best_frames]
    if use_threshhold:
        temp_frame_scores = []
        for frame_score in frame_scores:
            if frame_score[1] < threshhold:
                temp_frame_scores.append(frame_score)
        frame_scores = temp_frame_scores
    return frame_scores

In [6]:
column_name = list(template_data_df.columns)
frame_matrix = template_data_df.to_numpy()

In [7]:
sample_data_matrix = sample_data_df.to_numpy()
row_length = len(sample_data_matrix[0])
for i in range(row_length):
    print(i, sample_data_matrix[0][i])

0 1038.0330504115218
1 3223.163146219148
2 1178.4742530435535
3 3473.6491983882133
4 5658.779294195824
5 3614.090401020242
6 1133.97389135374
7 3319.103987161358
8 1274.415093985768
9 3680.403651191708
10 5865.533746999326
11 3820.844853823743
12 6116.019799168396
13 8301.14989497599
14 6256.461001800442
15 3776.344492133921
16 5961.474587941549
17 3916.7856947659448
18 1095.0938518089854
19 3280.2239476166164
20 1235.5350544410187
21 3530.709999785691
22 5715.840095593299
23 3671.151202417705
24 1191.0346927511994
25 3376.164788558828
26 1331.4758953832288
27 10484.0
28 436.8333333333333
29 temp_imgs\A008_01210855_C001_Proxy\0000.png
30 00:07:16.833
31 5242.0
32 0.0
33 0.0
34 C:\Users\Simon\git\Machine-Learning-And-Art\dataset_generator\temp_imgs\A008_01210855_C001_Proxy\0000.png
35 3.0
36 0.0
37 3.0
38 0.0
39 0.0
40 0.0
41 0.0
42 1.0
43 0.0
44 0.0
45 0.0
46 0.0
47 0.0
48 0.0
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 0.0
55 0.0
56 0.0
57 0.0
58 0.0
59 0.0
60 0.0
61 1.0
62 0.0
63 0.0
64 0.

In [8]:
new_frames = []
img_location_col= 29
frame_count = len(frame_matrix)
for i in range(frame_count):
    data = best_frame_finder(frame_matrix[i], sample_data_df, 5, auto_sort=True)
    new_frames.append([i,sample_data_matrix[data[0][0]][img_location_col]])
    print(new_frames[-1])

[0, 'temp_imgs\\A008_01212036_C052_Proxy\\0000.png']
[1, 'temp_imgs\\A032_02021116_C116_Proxy\\0000.png']
[2, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[3, 'temp_imgs\\A011_12010327_C027_Proxy\\0000.png']
[4, 'temp_imgs\\A009_11220915_C053_Proxy\\0000.png']
[5, 'temp_imgs\\A011_12010310_C018_Proxy\\0000.png']
[6, 'temp_imgs\\A009_11220847_C037_Proxy\\0000.png']
[7, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[8, 'temp_imgs\\A009_11220847_C037_Proxy\\0000.png']
[9, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[10, 'temp_imgs\\A009_11220915_C053_Proxy\\0000.png']
[11, 'temp_imgs\\A011_12010323_C023_Proxy\\0000.png']
[12, 'temp_imgs\\A032_02021116_C116_Proxy\\0000.png']
[13, 'temp_imgs\\A011_12010310_C018_Proxy\\0000.png']
[14, 'temp_imgs\\A030_02020101_C152_Proxy\\0000.png']
[15, 'temp_imgs\\A011_12010312_C019_Proxy\\0000.png']
[16, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[17, 'temp_imgs\\A032_02021116_C116_Proxy\\0000.png']
[18, 'temp_imgs\\A011_12010318_C022_Pr

[155, 'temp_imgs\\A030_02020055_C141_Proxy\\0000.png']
[156, 'temp_imgs\\A030_02010503_C062_Proxy\\0000.png']
[157, 'temp_imgs\\A032_02020403_C035_Proxy\\0000.png']
[158, 'temp_imgs\\A032_02020405_C042_Proxy\\0000.png']
[159, 'temp_imgs\\A030_02020055_C141_Proxy\\0000.png']
[160, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[161, 'temp_imgs\\A030_02010448_C058_Proxy\\0000.png']
[162, 'temp_imgs\\A010_01201606_C120_Proxy\\0000.png']
[163, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[164, 'temp_imgs\\A010_01201606_C120_Proxy\\0000.png']
[165, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[166, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[167, 'temp_imgs\\A030_02020101_C152_Proxy\\0000.png']
[168, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[169, 'temp_imgs\\A010_01201606_C120_Proxy\\0000.png']
[170, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[171, 'temp_imgs\\A011_12010312_C019_Proxy\\0000.png']
[172, 'temp_imgs\\A011_12010310_C018_Proxy\\0000.png']
[173, 'tem

[306, 'temp_imgs\\A030_02010503_C061_Proxy\\0000.png']
[307, 'temp_imgs\\A030_02010633_C073_Proxy\\0000.png']
[308, 'temp_imgs\\A032_02020407_C043_Proxy\\0000.png']
[309, 'temp_imgs\\A030_02010634_C075_Proxy\\0000.png']
[310, 'temp_imgs\\A030_02010725_C089_Proxy\\0000.png']
[311, 'temp_imgs\\A030_02010503_C061_Proxy\\0000.png']
[312, 'temp_imgs\\A032_02020958_C100_Proxy\\0000.png']
[313, 'temp_imgs\\A030_02010927_C105_Proxy\\0000.png']
[314, 'temp_imgs\\A030_02010634_C075_Proxy\\0000.png']
[315, 'temp_imgs\\A030_02010633_C073_Proxy\\0000.png']
[316, 'temp_imgs\\A030_02010633_C074_Proxy\\0000.png']
[317, 'temp_imgs\\A030_02010633_C074_Proxy\\0000.png']
[318, 'temp_imgs\\A030_02010633_C074_Proxy\\0000.png']
[319, 'temp_imgs\\A030_02010633_C073_Proxy\\0000.png']
[320, 'temp_imgs\\A030_02010634_C075_Proxy\\0000.png']
[321, 'temp_imgs\\A030_02010634_C075_Proxy\\0000.png']
[322, 'temp_imgs\\A008_01212028_C027_Proxy\\0000.png']
[323, 'temp_imgs\\A012_11291942_C039_Proxy\\0000.png']
[324, 'tem

[457, 'temp_imgs\\A010_01201549_C103_Proxy\\0000.png']
[458, 'temp_imgs\\A010_01201952_C138_Proxy\\0000.png']
[459, 'temp_imgs\\A010_01201942_C122_Proxy\\0000.png']
[460, 'temp_imgs\\A010_01201945_C125_Proxy\\0000.png']
[461, 'temp_imgs\\A010_01201945_C126_Proxy\\0000.png']
[462, 'temp_imgs\\A010_01202001_C153_Proxy\\0000.png']
[463, 'temp_imgs\\A009_11220905_C047_Proxy\\0000.png']
[464, 'temp_imgs\\A010_01201603_C116_Proxy\\0000.png']
[465, 'temp_imgs\\A009_11220905_C047_Proxy\\0000.png']
[466, 'temp_imgs\\A010_01201603_C116_Proxy\\0000.png']
[467, 'temp_imgs\\A009_11220912_C051_Proxy\\0000.png']
[468, 'temp_imgs\\A010_01201603_C116_Proxy\\0000.png']
[469, 'temp_imgs\\A009_11220912_C051_Proxy\\0000.png']
[470, 'temp_imgs\\A032_02021001_C107_Proxy\\0000.png']
[471, 'temp_imgs\\A008_01212040_C055_Proxy\\0000.png']
[472, 'temp_imgs\\A032_02021002_C109_Proxy\\0000.png']
[473, 'temp_imgs\\A032_02021133_C121_Proxy\\0000.png']
[474, 'temp_imgs\\A032_02021002_C109_Proxy\\0000.png']
[475, 'tem

[612, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[613, 'temp_imgs\\A011_12010312_C019_Proxy\\0000.png']


In [9]:
for new_frame in new_frames:
    input_path = f'{path}\\dataset_generator\\{new_frame[1]}'
    output_path = f'{path}\\timeline_generation\\temp\\imgs\\{str(new_frame[0]).rjust(4, "0")}.png'
    print(new_frame)
    im = Image.open(input_path)
    im = im.resize((1920, 1080))
    im.save(output_path)
    shutil.copyfile(input_path, output_path)

[0, 'temp_imgs\\A008_01212036_C052_Proxy\\0000.png']
[1, 'temp_imgs\\A032_02021116_C116_Proxy\\0000.png']
[2, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[3, 'temp_imgs\\A011_12010327_C027_Proxy\\0000.png']
[4, 'temp_imgs\\A009_11220915_C053_Proxy\\0000.png']
[5, 'temp_imgs\\A011_12010310_C018_Proxy\\0000.png']
[6, 'temp_imgs\\A009_11220847_C037_Proxy\\0000.png']
[7, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[8, 'temp_imgs\\A009_11220847_C037_Proxy\\0000.png']
[9, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[10, 'temp_imgs\\A009_11220915_C053_Proxy\\0000.png']
[11, 'temp_imgs\\A011_12010323_C023_Proxy\\0000.png']
[12, 'temp_imgs\\A032_02021116_C116_Proxy\\0000.png']
[13, 'temp_imgs\\A011_12010310_C018_Proxy\\0000.png']
[14, 'temp_imgs\\A030_02020101_C152_Proxy\\0000.png']
[15, 'temp_imgs\\A011_12010312_C019_Proxy\\0000.png']
[16, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[17, 'temp_imgs\\A032_02021116_C116_Proxy\\0000.png']
[18, 'temp_imgs\\A011_12010318_C022_Pr

[154, 'temp_imgs\\A008_11302109_C040_Proxy\\0000.png']
[155, 'temp_imgs\\A030_02020055_C141_Proxy\\0000.png']
[156, 'temp_imgs\\A030_02010503_C062_Proxy\\0000.png']
[157, 'temp_imgs\\A032_02020403_C035_Proxy\\0000.png']
[158, 'temp_imgs\\A032_02020405_C042_Proxy\\0000.png']
[159, 'temp_imgs\\A030_02020055_C141_Proxy\\0000.png']
[160, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[161, 'temp_imgs\\A030_02010448_C058_Proxy\\0000.png']
[162, 'temp_imgs\\A010_01201606_C120_Proxy\\0000.png']
[163, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[164, 'temp_imgs\\A010_01201606_C120_Proxy\\0000.png']
[165, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[166, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[167, 'temp_imgs\\A030_02020101_C152_Proxy\\0000.png']
[168, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[169, 'temp_imgs\\A010_01201606_C120_Proxy\\0000.png']
[170, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[171, 'temp_imgs\\A011_12010312_C019_Proxy\\0000.png']
[172, 'tem

[304, 'temp_imgs\\A030_02010503_C061_Proxy\\0000.png']
[305, 'temp_imgs\\A030_02010219_C030_Proxy\\0000.png']
[306, 'temp_imgs\\A030_02010503_C061_Proxy\\0000.png']
[307, 'temp_imgs\\A030_02010633_C073_Proxy\\0000.png']
[308, 'temp_imgs\\A032_02020407_C043_Proxy\\0000.png']
[309, 'temp_imgs\\A030_02010634_C075_Proxy\\0000.png']
[310, 'temp_imgs\\A030_02010725_C089_Proxy\\0000.png']
[311, 'temp_imgs\\A030_02010503_C061_Proxy\\0000.png']
[312, 'temp_imgs\\A032_02020958_C100_Proxy\\0000.png']
[313, 'temp_imgs\\A030_02010927_C105_Proxy\\0000.png']
[314, 'temp_imgs\\A030_02010634_C075_Proxy\\0000.png']
[315, 'temp_imgs\\A030_02010633_C073_Proxy\\0000.png']
[316, 'temp_imgs\\A030_02010633_C074_Proxy\\0000.png']
[317, 'temp_imgs\\A030_02010633_C074_Proxy\\0000.png']
[318, 'temp_imgs\\A030_02010633_C074_Proxy\\0000.png']
[319, 'temp_imgs\\A030_02010633_C073_Proxy\\0000.png']
[320, 'temp_imgs\\A030_02010634_C075_Proxy\\0000.png']
[321, 'temp_imgs\\A030_02010634_C075_Proxy\\0000.png']
[322, 'tem

[454, 'temp_imgs\\A010_01201952_C138_Proxy\\0000.png']
[455, 'temp_imgs\\A010_01202001_C153_Proxy\\0000.png']
[456, 'temp_imgs\\A010_01201942_C122_Proxy\\0000.png']
[457, 'temp_imgs\\A010_01201549_C103_Proxy\\0000.png']
[458, 'temp_imgs\\A010_01201952_C138_Proxy\\0000.png']
[459, 'temp_imgs\\A010_01201942_C122_Proxy\\0000.png']
[460, 'temp_imgs\\A010_01201945_C125_Proxy\\0000.png']
[461, 'temp_imgs\\A010_01201945_C126_Proxy\\0000.png']
[462, 'temp_imgs\\A010_01202001_C153_Proxy\\0000.png']
[463, 'temp_imgs\\A009_11220905_C047_Proxy\\0000.png']
[464, 'temp_imgs\\A010_01201603_C116_Proxy\\0000.png']
[465, 'temp_imgs\\A009_11220905_C047_Proxy\\0000.png']
[466, 'temp_imgs\\A010_01201603_C116_Proxy\\0000.png']
[467, 'temp_imgs\\A009_11220912_C051_Proxy\\0000.png']
[468, 'temp_imgs\\A010_01201603_C116_Proxy\\0000.png']
[469, 'temp_imgs\\A009_11220912_C051_Proxy\\0000.png']
[470, 'temp_imgs\\A032_02021001_C107_Proxy\\0000.png']
[471, 'temp_imgs\\A008_01212040_C055_Proxy\\0000.png']
[472, 'tem

[604, 'temp_imgs\\DJI_0129_Proxy\\0000.png']
[605, 'temp_imgs\\A032_02021002_C108_Proxy\\0000.png']
[606, 'temp_imgs\\A011_12011514_C002_Proxy\\0000.png']
[607, 'temp_imgs\\A030_02010418_C051_Proxy\\1119.png']
[608, 'temp_imgs\\A010_01201606_C120_Proxy\\0000.png']
[609, 'temp_imgs\\A030_02020101_C152_Proxy\\0000.png']
[610, 'temp_imgs\\A010_01201605_C119_Proxy\\0000.png']
[611, 'temp_imgs\\A011_12010318_C022_Proxy\\0000.png']
[612, 'temp_imgs\\A011_12010328_C029_Proxy\\0000.png']
[613, 'temp_imgs\\A011_12010312_C019_Proxy\\0000.png']


In [10]:
frame_rate = 10
timestamp = str(time.time()).split(".")[0]
input_path = f'{path}\\timeline_generation\\temp\\imgs\\%04d.png'
output_path = f'{path}\\timeline_generation\\temp\\output\\movie_{timestamp}.mp4'
cmd = f'ffmpeg -r {frame_rate} -f image2 -s 1920x1080 -i \"{input_path}\" -vcodec libx264 -crf 25  -pix_fmt yuv420p \"{output_path}\"'
print(cmd)
subprocess.run(cmd, shell=True)

ffmpeg -r 10 -f image2 -s 1920x1080 -i "C:\Users\Simon\git\Machine-Learning-And-Art\timeline_generation\temp\imgs\%04d.png" -vcodec libx264 -crf 25  -pix_fmt yuv420p "C:\Users\Simon\git\Machine-Learning-And-Art\timeline_generation\temp\output\movie_1614322423.mp4"


CompletedProcess(args='ffmpeg -r 10 -f image2 -s 1920x1080 -i "C:\\Users\\Simon\\git\\Machine-Learning-And-Art\\timeline_generation\\temp\\imgs\\%04d.png" -vcodec libx264 -crf 25  -pix_fmt yuv420p "C:\\Users\\Simon\\git\\Machine-Learning-And-Art\\timeline_generation\\temp\\output\\movie_1614322423.mp4"', returncode=0)